In [1]:
import glob
import cupy as cp
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import soundfile as sf
import gc

In [2]:
trainfiles = glob.glob( 'data/train/*.flac' )
testfiles = glob.glob( 'data/test/*.flac' )
len(trainfiles), len(testfiles), trainfiles[0]

(4727, 1992, 'data/train\\00204008d.flac')

In [3]:
traint = pd.read_csv( 'data/train_tp.csv' )
trainf = pd.read_csv( 'data/train_fp.csv' )

In [4]:
def extract_fft(fn):
    data, samplerate = sf.read(fn)
    data = cp.array(data)
    varfft = cp.abs( cp.fft.fft(data)[:(len(data)//2)] )
    return cp.asnumpy( varfft.reshape( (1000,1440) ).mean(axis=1) )

In [5]:
X_train = []
for fn in tqdm(trainfiles):
    X_train.append( extract_fft(fn) )
X_train = np.stack(X_train)
gc.collect()

X_train.shape

(4727, 1000)

In [6]:
X_test = []
for fn in tqdm(testfiles):
    X_test.append( extract_fft(fn) )
X_test = np.stack(X_test)
gc.collect()

X_test.shape

(1992, 1000)

In [7]:

# y_train = include labels for true positives and false positives
tt = traint[['recording_id','species_id']].copy()
tf = trainf[['recording_id','species_id']].copy()
tt["tp_and_fp"] = "tp"
tf["tp_and_fp"] = "fp"
# The order is True Positive first, False Positive second
y_train_all_classes = pd.concat( (tt, tf) )

for i in range(24):
    y_train_all_classes['s'+str(i)] = 0
    # Notice that the False Positive labels should be 0
    # Will correct them in the helper function
    y_train_all_classes.loc[y_train_all_classes.species_id==i,'s'+str(i)] = 1

y_train_all_classes.tail()

,recording_id,species_id,tp_and_fp,s0,s1,s2,s3,s4,s5,s6,...,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
7776,ffd88cd84,14,fp,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7777,ffebe7313,1,fp,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7778,fff163132,17,fp,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7779,fff163132,14,fp,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7780,fffb79246,6,fp,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X_train = pd.DataFrame(X_train)
recording_id = [path[44:53] for path in trainfiles]
X_train = X_train.set_index(pd.Index(recording_id))

X_test = pd.DataFrame(X_test)

In [9]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
,11.790391,35.339951,33.839856,24.667511,19.069494,15.649259,15.250774,15.013724,13.815972,12.297996,...,2.239256,2.334394,2.206936,2.271055,2.221637,2.324264,2.284480,2.319976,2.269804,2.315348
,23.789466,41.844881,34.087390,24.990900,33.028959,29.955412,39.047059,32.389619,25.473174,23.336571,...,3.865874,3.812509,3.811468,3.771802,3.760778,3.813280,3.733179,3.802968,3.776638,3.893550
,10.672936,21.634743,33.368987,27.741980,63.404136,19.311545,22.348948,15.599358,32.488332,19.724591,...,3.154830,3.184257,3.164174,3.174290,3.087597,2.892178,3.010763,3.027793,2.996881,3.074568
,227.698311,277.244232,394.248199,141.685471,178.871626,247.544134,157.892434,274.957322,230.505231,229.685355,...,5.692819,7.393680,6.652094,6.163386,6.314197,6.556996,6.150115,6.168900,5.961046,5.587845
,97.733087,207.211464,217.810163,178.775455,296.608300,331.616375,206.194665,133.995695,99.661902,62.896095,...,2.363506,2.374879,2.395297,2.411069,2.405776,2.426955,2.392858,2.413523,2.325952,2.367381


In [10]:
X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,26.777072,48.789862,48.658318,38.345478,29.403780,22.604672,21.870240,22.177086,23.431294,19.994830,...,2.844235,2.904594,2.859302,2.780773,2.749330,2.705327,2.715763,2.707143,2.663817,2.696315
1,15.370870,134.196483,137.053862,86.158029,119.924272,102.650543,116.111480,270.525276,275.029161,189.287502,...,2.894080,2.900234,2.911039,2.951832,2.927140,2.856842,2.900470,2.921184,3.007650,2.850644
2,102.291469,146.259771,184.226037,87.693767,52.898082,51.871333,46.215581,58.855169,63.327388,84.239141,...,3.613505,3.624661,3.666196,3.659059,3.619957,3.654109,3.714032,3.708906,3.647022,3.700800
3,41.571906,50.391221,52.416428,47.872607,43.501722,41.260662,43.912675,48.554317,37.681341,33.533002,...,11.037309,11.684867,11.904491,12.397820,13.009769,13.343707,13.792227,14.027738,14.297619,14.354401
4,9.232961,26.556042,20.103448,17.712726,17.324920,12.793497,12.460659,12.131253,11.745227,11.762307,...,3.470543,3.545655,3.652730,3.485103,3.645361,3.431659,3.618027,3.526598,3.630001,3.662966


In [12]:
X_train.to_hdf("savefiles/CNN/X_train.hd5", key="stage", mode='w')
X_test.to_hdf("savefiles/CNN/X_test.hd5", key="stage", mode='w')
y_train_all_classes.to_hdf("savefiles/CNN/y_train.hd5", key="stage", mode='w')